<a href="https://colab.research.google.com/github/olonok69/projects_ai/blob/main/CSM_generate_Embeddings_badges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -U bitsandbytes Faker -q

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import json
import gc
import datetime
from faker import Faker

fake = Faker()
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import json
from tqdm import tqdm

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
session_data_path = "/content/drive/MyDrive/data/CSM/speaker_aggregated_info.json"  # For Recommendation engine
session_data_path = "/content/drive/MyDrive/data/CSM/badge_id_aggregated_results.json"
nomenclature_embeddings_path = (
    "/content/drive/MyDrive/data/CSM/cluster_numeculature.json"
)
output_path = "/content/drive/MyDrive/data/CSM/badge/embeddings_badges_mistral.json"

In [ ]:
with open(session_data_path) as f:
    session_data = json.load(f)

In [ ]:
session_data["BadgeId"][0]["BadgeId"]

In [ ]:
session_data["BadgeId"][1]["BadgeId"]

In [ ]:
session_data["BadgeId"][100].keys()

In [ ]:
for key, values_list in session_data.items():
    print(key, len(values_list))
    break

In [ ]:
len(session_data["BadgeId"])

In [ ]:
max_length = 0
for value_dict in session_data["BadgeId"]:
    value_len = len(value_dict.get("AggregatedInfo"))
    if value_len > max_length:
        max_length = value_len
print(max_length)

In [ ]:
def generate_session_embeddings_all_json(
    session_data_path, output_file_path, tokenizer, model, unique_key: str = "BadgeId"
):
    # Assuming the necessary imports and model initialization are done above this method.

    with open(session_data_path) as f:
        session_data = json.load(f)

    embeddings = {}
    for key, values_list in session_data.items():
        for value_dict in tqdm(
            values_list, desc="Generating session embeddings"
        ):  # Iterate over each item in the list associated with the key
            aggregated_info = value_dict.get(
                "AggregatedInfo"
            )  # Extract the AggregatedInfo
            badgeid = value_dict.get("BadgeId")
            # print(badgeid)
            # print(key, value_dict)
            if aggregated_info:  # Check if AggregatedInfo exists

                inputs = tokenizer(
                    aggregated_info,
                    return_tensors="pt",
                    truncation=True,
                    max_length=512,
                )
                inputs = inputs.to("cuda")
                # Inputs are now based on the 'AggregatedInfo' text

                with torch.no_grad():
                    outputs = model(**inputs, output_hidden_states=True)

                hidden_states = outputs.hidden_states
                second_to_last_layer = hidden_states[-2]
                # Assuming each BadgeId is unique, combine key and BadgeId for a unique identifier
                unique_key = f"{key}_{value_dict.get('BadgeId', 'unkown_session_id')}"
                # print(unique_key)
                embeddings[unique_key] = second_to_last_layer.mean(dim=1).cpu().numpy()

    # Save embeddings to file
    with open(output_file_path, "w", encoding="utf-8") as f:
        json.dump(
            {k: v.tolist() for k, v in embeddings.items()},
            f,
            ensure_ascii=False,
            indent=4,
        )

    return embeddings

# Mistral

https://huggingface.co/mistralai/Mistral-7B-v0.1

Embeddings 4096

Mistral 7B: This model has a context window size of 32k with default of 8,000 tokens

## Time: 2:57:02.006613 Badge Embeddings 2024

In [ ]:
model_id = "mistralai/Mistral-7B-v0.1"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, device_map="auto", load_in_8bit=True)

In [ ]:
time_start = datetime.datetime.now()
embeddings_mistral = generate_session_embeddings_all_json(
    session_data_path, output_path, tokenizer, model, unique_key="BadgeId"
)
time_end = datetime.datetime.now()
print(f"Time taken: {time_end - time_start}")

In [ ]:
len(embeddings_mistral.keys())

In [ ]:
embeddings_mistral["BadgeId_22A5D99"]

In [ ]:
keys = ["BadgeId_22A5D99", "BadgeId_22DIQKA"]

In [ ]:
len(embeddings_mistral["BadgeId_22A5D99"][0]), len(
    embeddings_mistral["BadgeId_22DIQKA"][0]
)

In [ ]:
def cosine_distance(A, B):
    dot_product = np.dot(A, B)
    magnitude_A = np.linalg.norm(A)
    magnitude_B = np.linalg.norm(B)
    return 1 - dot_product / (magnitude_A * magnitude_B)

In [ ]:
cosine_distance(
    embeddings_mistral["BadgeId_22A5D99"][0], embeddings_mistral["BadgeId_22DIQKA"][0]
)

In [ ]:
del model

del tokenizer
torch.cuda.empty_cache()
gc.collect()

In [ ]:
torch.cuda.empty_cache()
gc.collect()

# LLama 3.2B 3B
https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct

embeddings length 3072

Llama 3.2 3B: This model supports a context window size of up to 128,000 tokens, with a default setting of 8,192 tokens


# Time: 1:00:07 Badge Embeddings 2024

In [ ]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, device_map="auto", load_in_8bit=True)

In [ ]:
output_path = "/content/drive/MyDrive/data/CSM/badge/embeddings_badges_llama.json"

In [ ]:
time_start = datetime.datetime.now()
embeddings_llama = generate_session_embeddings_all_json(
    session_data_path, output_path, tokenizer, model
)
time_end = datetime.datetime.now()
print(f"Time taken: {time_end - time_start}")

In [ ]:
len(embeddings_llama["BadgeId_22A5D99"][0]), len(embeddings_llama["BadgeId_22DIQKA"][0])

In [ ]:
cosine_distance(
    embeddings_llama["BadgeId_22A5D99"][0], embeddings_llama["BadgeId_22DIQKA"][0]
)

In [ ]:
del model

del tokenizer
torch.cuda.empty_cache()
gc.collect()

In [ ]:
torch.cuda.empty_cache()
gc.collect()

# NOMIC-embed-text-v1

https://huggingface.co/nomic-ai/nomic-embed-text-v1

Length 768 MAX

8192 context length text encoder


## GPU Time: 0:33:31 Badge Embeddings 2024

## CPU Time: 8:22:23 Badge Embeddings 2024

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

In [ ]:
output_path = "/content/drive/MyDrive/data/CSM/badge/embeddings_badges_nomic.json"

In [ ]:
# GPU
embeddings_nomic = {}
time_start = datetime.datetime.now()
for key, values_list in session_data.items():
    for value_dict in tqdm(
        values_list, desc="Generating session embeddings"
    ):  # Iterate over each item in the list associated with the key
        aggregated_info = value_dict.get("AggregatedInfo")
        badgeid = value_dict.get("BadgeId")
        if aggregated_info:
            # Assuming each BadgeId is unique, combine key and BadgeId for a unique identifier
            unique_key = f"{key}_{value_dict.get('BadgeId', 'unkown_session_id')}"
            embeddings_nomic[unique_key] = model.encode(
                f"search_document: {aggregated_info}"
            )

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(
        {k: v.tolist() for k, v in embeddings_nomic.items()},
        f,
        ensure_ascii=False,
        indent=4,
    )

time_end = datetime.datetime.now()
print(f"Time taken: {time_end - time_start}")

In [ ]:
cosine_distance(
    embeddings_nomic["BadgeId_22A5D99"], embeddings_nomic["BadgeId_22DIQKA"]
)

In [ ]:
del model

# del tokenizer
torch.cuda.empty_cache()
gc.collect()

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
with open(output_path) as f:
    embeddings_nomic = json.load(f)

In [ ]:
  len(embeddings_nomic.keys())


In [ ]:
len(embeddings_nomic["BadgeId_22A5D99"])

In [ ]:
cosine_distance(
    embeddings_nomic["BadgeId_22A5D99"], embeddings_nomic["BadgeId_22DIQKA"]
)

In [ ]:
output_path = "/content/drive/MyDrive/data/CSM/badge/embeddings_badges_nomic_cpu.json"

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(
    "nomic-ai/nomic-embed-text-v1", trust_remote_code=True, device="cpu"
)

In [ ]:
# CPU
embeddings_nomic_cpu = {}
time_start = datetime.datetime.now()
for key, values_list in session_data.items():
    for value_dict in tqdm(
        values_list, desc="Generating session embeddings"
    ):  # Iterate over each item in the list associated with the key
        aggregated_info = value_dict.get("AggregatedInfo")
        if aggregated_info:
            # Assuming each BadgeId is unique, combine key and BadgeId for a unique identifier
            unique_key = f"{key}_{value_dict.get('BadgeId', 'unkown_session_id')}"
            embeddings_nomic_cpu[unique_key] = model.encode(
                f"search_document: {aggregated_info}"
            )

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(
        {k: v.tolist() for k, v in embeddings_nomic_cpu.items()},
        f,
        ensure_ascii=False,
        indent=4,
    )

time_end = datetime.datetime.now()
print(f"Time taken: {time_end - time_start}")

In [ ]:
cosine_distance(
    embeddings_nomic["BadgeId_22A5D99"], embeddings_nomic["BadgeId_22DIQKA"]
)